In [1]:
%load_ext autoreload
%autoreload 2

# Comparison of mutational robustness of sRNA's vs. randomly generated RNA

## Imports

In [2]:
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re
import os
from subprocess import Popen, PIPE
from copy import deepcopy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from synbio_morpher.srv.io.manage.script_manager import script_preamble
from synbio_morpher.srv.parameter_prediction.simulator import process_raw_stdout
from synbio_morpher.srv.sequence_exploration.sequence_analysis import b_tabulate_mutation_info
from synbio_morpher.utils.common.setup import construct_circuit_from_cfg, prepare_config, expand_config
from synbio_morpher.utils.evolution.evolver import Evolver
from synbio_morpher.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller
from synbio_morpher.utils.misc.type_handling import flatten_listlike, flatten_nested_dict
from synbio_morpher.utils.data.data_format_tools.common import load_multiple_as_list, load_json_as_dict


import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')

jax.devices()

/home/wadh6511/Kode/env_evo/lib/python3.10/site-packages/jaxtyping/__init__.py:221: UserWarning: jaxtyping version >=0.2.23 should be used with Equinox version >=0.11.1
  warnings.warn(
I0000 00:00:1704821888.438250  425322 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.
xla_bridge.py:backends():513: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA INFO
xla_bridge.py:backends():513: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory INFO


[gpu(id=0)]

## Load data

In [3]:
fn = '../data/sRNA/merged_inter.csv'
data = pd.read_csv(fn)
try:
    data = data.drop(columns=['Unnamed: 0'])
except:
    pass
data = data[data['Sequence 1'].apply(lambda x: type(x) == str)]
data = data[data['Sequence 2'].apply(lambda x: type(x) == str)]


In [4]:
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB,Genome position 1,Genome position 2,Strand 1,Strand 2,Sequence 1 before,Sequence 2 before,Sequence 1 after,Sequence 2 after
0,micA,dppA,NCBI:2847697,NCBI:948062,sRNA,mRNA,NaN,NaN,NaN,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGCGTATTTCCTTGAAAAAGTCAGGGATGCTGAAGCTTGGTCTCA...,RNAInter,NaN,NaN,NaN,NaN,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,attattctgctccaattgtgatgtttgttgttttaaccctttgcag...,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,attattctgctccaattgtgatgtttgttgttttaaccctttgcag...
1,dsrA,dsrB,NCBI:946470,NCBI:946468,sRNA,mRNA,NaN,NaN,NaN,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,TAAATGNAAACAAGTTTTATTTTTATCATCAGAACATTCTAATAAA...,RNAInter,NaN,NaN,NaN,NaN,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,atcgttttcctcctgtggctttgtgccagtgtagaacaatttcgtt...,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,atcgttttcctcctgtggctttgtgccagtgtagaacaatttcgtt...
2,dsrA,ilvH,NCBI:946470,NCBI:947267,sRNA,mRNA,NaN,NaN,NaN,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,RNAInter,NaN,NaN,NaN,NaN,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...
3,micA,ecnB,NCBI:2847697,NCBI:2847737,sRNA,mRNA,NaN,NaN,NaN,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGGTGAAGAAGACAATTGCAGCGATCTTTTCTGTTCTGGTGCTTT...,RNAInter,NaN,NaN,NaN,NaN,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,CTTTTGCGCCTCGCTTCGACGTCCGCGAATCCAACGAGGCCTACCA...,CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,CTTTTGCGCCTCGCTTCGACGTCCGCGAATCCAACGAGGCCTACCA...
4,oxyS,fhlA,NCBI:2847701,NCBI:947181,sRNA,mRNA,NaN,NaN,NaN,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,RNAInter,NaN,NaN,NaN,NaN,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",sRNA,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase,3580927..3581016,4441538..4442176,reverse,reverse,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,atggtgtcgctctcccgaaaatcggtcattggggttaaggttgtaa...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,atggtgtcgctctcccgaaaatcggtcattggggttaaggttgtaa...
443,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase,2942696..2942901,3656408..3656740,forward,reverse,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,atcgtaatatcctcaactataaagtgaaagagccgtcacgaatcaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,atcgtaatatcctcaactataaagtgaaagagccgtcacgaatcaa...
444,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase,2942696..2942901,3655966..3656292,forward,reverse,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,attttgtaacccattcaatatagagattatatatgaacaggaggaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,attttgtaacccattcaatatagagattatatatgaacaggaggaa...
445,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGT

In [5]:
len(data)

446

# Config

In [6]:
config = {
    "experiment": {
        "purpose": "tests",
        "no_visualisations": False,
        "no_numerical": False,
        "debug_mode": False
    },
    "system_type": "RNA",
    "include_prod_deg": False,
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True,
        "simulator_kwargs": {
            "outcsvcols": "id1,id2,E,E_norm,subseqDB,Pu1,Pu2",
            "threads": 12,
            "n": 1,
            "raw_stdout": True
        }
    },
    "mutations_args": {
        "algorithm": "all",
        "mutation_counts": 0,
        "mutation_nums_within_sequence": [1],
        "mutation_nums_per_position": 1,
        "concurrent_species_to_mutate": "single_species_at_a_time",
        "seed": 0
    },
    "signal": {
        "inputs": [data['Interactor 1'].iloc[0]],
        "outputs": [],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 1,
            "impulse_halfwidth": 5,
            "target": 2
        }
    },
    "simulation": {
        "dt0": 0.1,
        "t0": 0,
        "t1": 100,
        "tmax": 2000,
        "solver": "diffrax",
        "use_batch_mutations": True,
        "interaction_factor": 1,
        "batch_size": 200,
        "max_circuits": 2000,
        "device": "cpu",
        "threshold_steady_states": 0.05,
        "use_rate_scaling": True
    },
    "molecular_params_factor": 1,
    "molecular_params": {
        "avg_mRNA_per_cell": 100,
        "cell_doubling_time": 1200,
        "creation_rate": 2.35,
        "starting_copynumbers": 200,
        "degradation_rate": 0.01175,
        "association_binding_rate": 1000000
    }
}



# Simulate

In [7]:
data_writer = None
config, data_writer = script_preamble(config, data_writer)
config_file = expand_config(config=config)
config_file = prepare_config(config_file)
config_file

{'experiment': {'purpose': 'tests',
  'no_visualisations': False,
  'no_numerical': False,
  'debug_mode': False},
 'system_type': 'RNA',
 'include_prod_deg': False,
 'interaction_simulator': {'name': 'IntaRNA',
  'postprocess': True,
  'simulator_kwargs': {'qidxpos0': 1,
   'tidxpos0': 1,
   'outcsvcols': 'id1,id2,E,E_norm,subseqDB,Pu1,Pu2',
   'threads': 12,
   'n': 1,
   'param_file': '/home/wadh6511/Kode/env_evo/lib/python3.10/site-packages/synbio_morpher/utils/common/configs/simulators/intaRNA_parameterFile.sh',
   'query': None,
   'target': None,
   'raw_stdout': True},
  'molecular_params': {'avg_mRNA_per_cell': 100,
   'cell_doubling_time': 1200,
   'creation_rate': 2.35,
   'starting_copynumbers': 200,
   'degradation_rate': 0.01175,
   'association_binding_rate': 1000000,
   'creation_rate_per_molecule': 3.5475152795e-09,
   'degradation_rate_per_molecule': 1.77375763975e-11,
   'association_binding_rate_per_molecule': 0.00150958097},
  'compute_by_filename': False},
 'mutat

In [20]:
def simulate_IntaRNA_local(query: dict,
                           targets: dict,
                           sim_kwargs={}):
    fn1 = data_writer.output(data=query, out_type='fasta',
                             out_name='query', byseq=True, return_path=True)
    fn2 = data_writer.output(data=targets, out_type='fasta',
                             out_name='targets', byseq=True, return_path=True)
    sim_kwargs['query'] = fn1
    sim_kwargs['target'] = fn2

    def run(query: str, target: str, qidxpos0: int, tidxpos0: int, outcsvcols: str, threads: int, n: int = 1,
            param_file: str = '', extra_params: list = [], raw_stdout: bool = False):
        p = Popen(['IntaRNA', '-q', query, '-t', target,
                   '--outMode=C', f'--outcsvcols={outcsvcols}',
                   f'--qIdxPos0={qidxpos0}',
                   f'--tIdxPos0={tidxpos0}',
                   f'--outNumber={n}',
                   f'--threads={threads}', param_file]
                  + extra_params, stdout=PIPE, stderr=PIPE, universal_newlines=True)
        stdout, stderr = p.communicate()
        return process_raw_stdout(stdout)

    return run(**sim_kwargs)


run_intarna = True

if run_intarna:
    
    sim_data = {}
    for sp in data.iterrows():
        t0 = datetime.now()
        i = sp[0]
        sp = sp[1]
        sim_data.setdefault(sp['Interactor 1'], {})
        sim_data[sp['Interactor 1']][sp['Interactor 2']] = simulate_IntaRNA_local(query={sp['Interactor 1']: sp['Sequence 1'].replace('T', 'U')},
                                            targets={sp['Interactor 2']: sp['Sequence 2'].replace('T', 'U')},
                                            sim_kwargs=config['interaction_simulator']['simulator_kwargs'])
        print(f'{i}\t Finished species {sp["Interactor 1"]} (len {len(sp["Sequence 1"])}) and {sp["Interactor 2"]} (len {len(sp["Sequence 2"])})\t in {datetime.now() - t0}')
    for k in sim_data:
        for kk, v in sim_data[k].items():
            l1 = v.get(k, v.get(kk))
            if l1:
                l1 = l1.get(k, l1.get(kk))
            sim_data[k][kk] = l1
    data_writer.output(data=sim_data, out_type='json',
                    out_name='inter_data_raw', overwrite=True)
    print(data_writer.write_dir)

0 Finished species micA and dppA in 0:00:12.374330
1 Finished species dsrA and dsrB in 0:00:00.119125
2 Finished species dsrA and ilvH in 0:00:00.142049
3 Finished species micA and ecnB in 0:00:00.649948
4 Finished species oxyS and fhlA in 0:00:00.139281
5 Finished species micA and fimB in 0:00:06.121611
6 Finished species micA and galK in 0:00:08.868418
7 Finished species micA and gloA in 0:00:02.329934
8 Finished species sgrS and hfq in 0:00:00.959252
9 Finished species micA and hns in 0:00:02.616550
10 Finished species micA and lamB in 0:00:11.719315
11 Finished species micA and ldtC in 0:00:07.488981
12 Finished species micA and lpxT in 0:00:06.268599
13 Finished species micA and luxS in 0:00:04.928160
15 Finished species micA and ompA in 0:00:06.460176
16 Finished species micA and ompC in 0:00:07.489077
17 Finished species micA and ompF in 0:00:09.463313
18 Finished species micA and ompT in 0:00:06.602618
19 Finished species micA and ompW in 0:00:06.130339
20 Finished species micA

In [ ]:
sim_data_l = load_json_as_dict(os.path.join(data_writer.write_dir, 'inter_data_raw.json'))
default_vals = {
    'id1': '', 'id2': '', 'E': 0.0, 'E_norm': 0.0, 'subseqDB': '', 'bpList': '', 'hybridDPfull': '', 'Pu1': 0.0, 'Pu2': 0.0, 'seedPu1': '', 'seedPu2': '', 'seedStart1': '', 'seedStart2': '', 'seedEnd1': '', 'seedEnd2': ''
}
for s, v in sim_data_l.items():
    for s2 in v:
        if v[s2] == {}:
            v[s2].update(default_vals)

## Mutate

Only mutate the parts of the sequence that are already binding, otherwise the whole simulation will take way too long.

First preprocess the binding sites curated from the interaction databases. If there are no documented binding 

In [128]:
pattern = r'\s*\([^)]*\)'
data['Binding position 1A'] = list(map(lambda raw: flatten_listlike(
    [[[int(re.sub(pattern, '', bp)) for bp in bps.split('..')] 
      for bps in re.sub(pattern, '', par).split('; ')] for par in raw if par.split('..')[0].strip('-').isnumeric()]), data['Binding position 1'].apply(str).str.split("'")))
data['Binding position 2A'] = list(map(lambda raw: flatten_listlike(
    [[[int(re.sub(pattern, '', bp)) for bp in bps.split('..')] 
      for bps in re.sub(pattern, '', par).split('; ')] for par in raw if par.split('..')[0].strip('-').isnumeric()]), data['Binding position 2'].apply(str).str.split("'")))
c = list(data.columns)
data = data[c[:-5] + c[-2:] + c[-5:-2]]
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Source DB,Binding position 1A,Binding position 2A,Sequence 1,Sequence 2
0,micA,dppA,NCBI:2847697,NCBI:948062,sRNA,mRNA,NaN,NaN,NaN,RNAInter,[],[],CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGCGTATTTCCTTGAAAAAGTCAGGGATGCTGAAGCTTGGTCTCA...
1,dsrA,dsrB,NCBI:946470,NCBI:946468,sRNA,mRNA,NaN,NaN,NaN,RNAInter,[],[],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,TAAATGNAAACAAGTTTTATTTTTATCATCAGAACATTCTAATAAA...
2,dsrA,ilvH,NCBI:946470,NCBI:947267,sRNA,mRNA,NaN,NaN,NaN,RNAInter,[],[],AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...
3,micA,ecnB,NCBI:2847697,NCBI:2847737,sRNA,mRNA,NaN,NaN,NaN,RNAInter,[],[],CAAAAATGGTGTCGCTCGTATTATTGTCGAGAAACCTTTCGGCAAG...,ATGGTGAAGAAGACAATTGCAGCGATCTTTTCTGTTCTGGTGCTTT...
4,oxyS,fhlA,NCBI:2847701,NCBI:947181,sRNA,mRNA,NaN,NaN,NaN,RNAInter,[],[],GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",NaN,mRNA,No Interaction,['NA'],['NA'],sRNATarBase,[],[],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...
445,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],sRNATarBase,[],[],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...
446,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],sRNATarBase,[],[],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...
447,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],sRNATarBase,[],[],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...


In [149]:
data['Binding position 2A'].apply(lambda x: np.min(np.array(x)) if x else None).min()

-131.0

In [16]:
circuits = []
for sp in data.iterrows():
    sp = sp[1]
    config_file['mutations_args']['concurrent_species_to_mutate'] = sp['Interactor 1']
    config_file['data'] = {
        sp['Interactor 1']: sp['Sequence 1'].replace('T', 'U'),
        sp['Interactor 2']: sp['Sequence 2'].replace('T', 'U')
    }
    circuit = construct_circuit_from_cfg(
        prev_configs=None, config_file=config_file)
    # circuit = Evolver(data_writer=data_writer, sequence_type=config_file.get('system_type'),
    #                   seed=config_file['mutations_args'].get('seed', 0),
    #                   concurrent_species_to_mutate=config_file['mutations_args']['concurrent_species_to_mutate']).mutate(
    #     circuit,
    #     write_to_subsystem=True, algorithm=config_file.get('mutations_args', {}).get('algorithm', 'random'))
    circuits.append(circuit)

len(circuit.model.species)

5

In [17]:
len(circuits)

446

In [18]:
# for circuit in circuits:
num_subcircuits = [len(flatten_nested_dict(
    c.mutations)) + 1 for c in circuits]
tot_subcircuits = sum(num_subcircuits)

print('Total circuits: ', tot_subcircuits)
print(num_subcircuits)

Total circuits:  205559
[1342, 262, 262, 1342, 331, 1342, 1342, 1342, 682, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 1342, 331, 271, 658, 3535, 271, 2485, 364, 262, 262, 262, 262, 262, 280, 328, 331, 421, 265, 265, 265, 265, 265, 247, 247, 247, 247, 265, 265, 265, 247, 247, 319, 274, 274, 238, 271, 238, 193, 271, 271, 271, 271, 271, 328, 262, 262, 262, 262, 271, 160, 682, 682, 316, 622, 553, 559, 559, 232, 169, 169, 166, 166, 265, 331, 331, 619, 265, 247, 199, 619, 271, 271, 271, 271, 271, 271, 271, 271, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 331, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 271, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 265, 247, 247, 247, 247, 247, 247, 247, 247, 247, 24

In [19]:
# from synbio_morpher.utils.common.setup import compose_kwargs
# modeller = CircuitModeller(result_writer=data_writer, config=config_file)
# curr_subcircuits = []
# for i, circuit in enumerate(circuits[:3]):
#     curr_subcircuits.append(modeller.load_mutations(circuit))
# curr_subcircuits = flatten_listlike(curr_subcircuits)


In [20]:
modeller = CircuitModeller(result_writer=data_writer, config=config_file)
circs = modeller.batch_circuits(
    circuits=circuits[:2],
    write_to_subsystem=True, batch_size=config_file['simulation'].get('batch_size', 100),
    methods={
        "compute_interactions": {},
        # "init_circuits": {'batch': True},
        # "simulate_signal_batch": {'ref_circuit': None,
        #                           'batch': True},
        "write_results": {'no_visualisations': config_file['experiment'].get('no_visualisations', True),
                          'no_numerical': config_file['experiment'].get('no_numerical', False)}
    })

circuit_manager.py:batch_circuits():640: 	From 2 circuits, a total of 1604 mutated circuits will be simulated. WARNING
circuit_manager.py:batch_circuits():647: 		Starting new round of viable circuits (0 - 2 / 2) WARNING
circuit_manager.py:batch_circuits():665: 		Making subcircuits 0 - 1604 took 1.499855s WARNING
circuit_manager.py:batch_circuits():671: 	Batching 0 - 200 circuits (out of 0 - 1604 (total: 1604)) (Circuits: 0 - 2 of 2) WARNING
circuit_manager.py:run_batch():703: 		Running 200 Subcircuits - circuit_1834054621: compute_interactions WARNING


KeyboardInterrupt: 

In [ ]:
circuits = []
for sp in data.iterrows():
    sp = sp[1]
    ref_circuit = {
        sp['Interactor 1']: sp['Sequence 1'].replace('T', 'U'),
        sp['Interactor 2']: sp['Sequence 2'].replace('T', 'U')
    }
    region = 
    
    config_file['mutations_args']['concurrent_species_to_mutate'] = sp['Interactor 1']
    # config_file['data_path'] = 'data/circuit.fasta'
    config_file['data'] = {
        sp['Interactor 1']: sp['Sequence 1'].replace('T', 'U'),
        sp['Interactor 2']: sp['Sequence 2'].replace('T', 'U')
    }
    circuit = construct_circuit_from_cfg(
        prev_configs=None, config_file=config_file)
    circuit = Evolver(data_writer=data_writer, sequence_type=config_file.get('system_type'),
                      seed=config_file['mutations_args'].get('seed', 0),
                      concurrent_species_to_mutate=config_file['mutations_args']['concurrent_species_to_mutate']).mutate(
        circuit,
        write_to_subsystem=True, algorithm=config_file.get('mutations_args', {}).get('algorithm', 'random'))
    circuits.append(circuit)

len(circuit.model.species)

## Save

In [ ]:
# source_dirs = [data_writer.top_write_dir]
# source_dirs = ['data/tests/2023_11_03_104039/']
# tables = load_multiple_as_list(inputs_list=source_dirs, load_func=b_tabulate_mutation_info, 
#             data_writer=data_writer)
# saves = pd.concat(tables, axis=0)
# saves

,circuit_name,mutation_name,mutation_num,mutation_type,mutation_positions,path_to_template_circuit,index,name,interacting,self_interacting,...,energies_1-0,energies_1-1,eqconstants_0-0,eqconstants_0-1,eqconstants_1-0,eqconstants_1-1,num_interacting_diff_to_base_circuit,num_self_interacting_diff_to_base_circuit,num_interacting_ratio_from_mutation_to_base,num_self_interacting_ratio_from_mutation_to_base
0,merged,ref_circuit,0,[],[],data/merged.fasta,0.0,merged,[],[[0 0]],...,0.00,0.0,1.170795,0.000002,0.000002,0.000002,0,0,0.0,1.0
1,merged,6S_RNA_m1-0,1,[2],[0],data/merged.fasta,0.0,merged,[],[[0 0]],...,0.00,0.0,1.170795,0.000002,0.000002,0.000002,0,0,0.0,1.0
2,merged,6S_RNA_m1-1,1,[1],[0],data/merged.fasta,1.0,merged,[],[[0 0]],...,0.00,0.0,1.124887,0.000002,0.000002,0.000002,0,0,0.0,1.0
3,merged,6S_RNA_m1-10,1,[9],[3],data/merged.fasta,10.0,merged,[],[[0 0]],...,0.00,0.0,0.322931,0.000002,0.000002,0.000002,0,0,0.0,1.0
4,merged,6S_RNA_m1-100,1,[3],[33],data/merged.fasta,100.0,merged,[],[[0 0]],...,0.00,0.0,0.174415,0.000002,0.000002,0.000002,0,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,merged,antisense_regulatory_RNA_RdlD_m1-95,1,[8],[31],data/merged.fasta,644.0,merged,[[0 1]],[[0 0]],...,-0.16,0.0,1.170795,0.000002,0.000002,0.000002,-1,0,0.0,1.0
738,merged,antisense_regulatory_RNA_RdlD_m1-96,1,[11],[32],data/merged.fasta,645.0,merged,[],[[0 0]],...,0.00,0.0,1.170795,0.000002,0.000002,0.000002,0,0,0.0,1.0
739,merged,antisense_regulatory_RNA_RdlD_m1-97,1,[9],[32],data/merged.fasta,646.0,merged,[],[[0 0]],...,0.00,0.0,1.170795,0.000002,0.000002,0.000002,0,0,0.0,1.0
740,merged,antisense_regulatory_RNA_RdlD_m1-98,1,[10],[32],data/merged.fasta,647.0,merged,[],[[0 0]],...,0.00,0.0,1.170795,0.000002,0.000002,0.000002,0,0,0.0,1.0


# Compare

In [ ]:
ddir = '/home/wadh6511/Kode/EvoScaper/notebooks_sRNA/data/04_distribution_mutational_robustness/2023_11_10_191219/mutations'
mutations = pd.read_csv('/home/wadh6511/Kode/EvoScaper/notebooks_sRNA/data/04_distribution_mutational_robustness/2023_11_10_191219/mutations.csv')

print(f'Number simulated: {len(os.listdir(ddir))}')
print(f'Percent simulated: {len(os.listdir(ddir)) / len(mutations) * 100} %')
print(f'Total number of mutations: {len(mutations)}')

Number simulated: 3363
Total number of mutations: 28821
